In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/Logistics Performance Index 2023.csv')
df

,Country,Year,LPI Rank,LPI Score,Customs Rank,Customs Score,Infrastructure Rank,Infrastructure Score,International shipments Rank,International shipments Score,Logistics competence Rank,Logistics competence Score,Tracking & tracing Rank,Tracking & tracing Score,Timeliness Rank,Timeliness Score
0,Afghanistan,2023,138,1.9,120,2.1,138,1.7,138,1.8,135,2.0,139,1.6,133,2.3
1,Albania,2023,97,2.5,90,2.4,68,2.7,75,2.8,119,2.3,117,2.3,124,2.5
2,Algeria,2023,97,2.5,101,2.3,125,2.1,57,3.0,126,2.2,98,2.5,116,2.6
3,Angola,2023,134,2.1,137,1.7,125,2.1,111,2.4,119,2.3,117,2.3,138,2.1
4,Antigua and Barbuda,2023,66,2.9,110,2.2,68,2.7,68,2.9,65,2.9,54,3.2,55,3.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,Uzbekistan,2023,88,2.6,74,2.6,89,2.4,91,2.6,92,2.6,105,2.4,101,2.8
135,"Venezuela, RB",2023,123,2.3,120,2.1,89,2.4,135,2.0,103,2.5,117,2.3,124,2.5
136,Vietnam,2023,43,3.3,43,3.1,47,3.2,38,3.3,53,3.2,41,3.4,59,3.3
137,"Yemen, Rep.",2023,132,2.2,137,1.7,132,1.9,139,1.7,92,2.6,117,2.3,101,2.8


### Normalization Step: Add ISO-alpha3 Code

In [ ]:
# --- Normalization: Add ISO-alpha3 column ---

# 1. Load UNSD Methodology for mapping
try:
    unsd_df = pd.read_csv('data/UNSD — Methodology.csv', sep=';')
except FileNotFoundError:
    unsd_df = pd.read_csv('../data/UNSD — Methodology.csv', sep=';')

# 2. Create mappings
unsd_df['Country or Area'] = unsd_df['Country or Area'].astype(str).str.strip()
name_to_iso3 = unsd_df.set_index('Country or Area')['ISO-alpha3 Code'].to_dict()

m49_to_iso3 = {}
for idx, row in unsd_df.iterrows():
    try:
        # Handle potential non-numeric or NaN M49 codes
        m49_code = int(row['M49 Code'])
        m49_to_iso3[m49_code] = row['ISO-alpha3 Code']
    except (ValueError, TypeError):
        continue

# 3. Apply mapping
print("Applying ISO-alpha3 normalization...")
if 'Country' in df.columns:
    df['iso_alpha3'] = df['Country'].astype(str).str.strip().map(name_to_iso3)
    print("  Mapped country names to 'iso_alpha3'.")
else:
    print("  Warning: Source column 'Country' not found.")

# Check results
missing_iso = df['iso_alpha3'].isna().sum()
if missing_iso > 0:
    print(f"  Warning: {missing_iso} rows have missing ISO-alpha3 codes.")
    print(df[df['iso_alpha3'].isna()][['Country']].head())


In [3]:
df = df.loc[:, ~df.columns.str.contains('Rank')]
df

,Country,Year,LPI Score,Customs Score,Infrastructure Score,International shipments Score,Logistics competence Score,Tracking & tracing Score,Timeliness Score
0,Afghanistan,2023,1.9,2.1,1.7,1.8,2.0,1.6,2.3
1,Albania,2023,2.5,2.4,2.7,2.8,2.3,2.3,2.5
2,Algeria,2023,2.5,2.3,2.1,3.0,2.2,2.5,2.6
3,Angola,2023,2.1,1.7,2.1,2.4,2.3,2.3,2.1
4,Antigua and Barbuda,2023,2.9,2.2,2.7,2.9,2.9,3.2,3.4
...,...,...,...,...,...,...,...,...,...
134,Uzbekistan,2023,2.6,2.6,2.4,2.6,2.6,2.4,2.8
135,"Venezuela, RB",2023,2.3,2.1,2.4,2.0,2.5,2.3,2.5
136,Vietnam,2023,3.3,3.1,3.2,3.3,3.2,3.4,3.3
137,"Yemen, Rep.",2023,2.2,1.7,1.9,1.7,2.6,2.3,2.8


In [6]:
df = df.iloc[:, :3]
df

,Country,Year,LPI Score
0,Afghanistan,2023,1.9
1,Albania,2023,2.5
2,Algeria,2023,2.5
3,Angola,2023,2.1
4,Antigua and Barbuda,2023,2.9
...,...,...,...
134,Uzbekistan,2023,2.6
135,"Venezuela, RB",2023,2.3
136,Vietnam,2023,3.3
137,"Yemen, Rep.",2023,2.2


In [7]:
df.rename(columns={'Country': 'Country Name'}, inplace=True)
df

,Country Name,Year,LPI Score
0,Afghanistan,2023,1.9
1,Albania,2023,2.5
2,Algeria,2023,2.5
3,Angola,2023,2.1
4,Antigua and Barbuda,2023,2.9
...,...,...,...
134,Uzbekistan,2023,2.6
135,"Venezuela, RB",2023,2.3
136,Vietnam,2023,3.3
137,"Yemen, Rep.",2023,2.2


In [ ]:
# ADD TO EVERY ROW OF FINAL FILE NO MATTER THE YEAR